In [1]:
import random
import json
from tokenizers import decoders, models, pre_tokenizers, trainers, Tokenizer
import os


random.seed(42)

In [7]:
path = "D:\\CODE\\train_tokenizer\\dataset\\tokenizer_training_data.jsonl"

def read_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            yield data['text']

In [8]:
texts = read_text(path)

In [9]:
#初始化Tokenizer
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)


#定义特殊的Token
special_tokens = ["<unk>", "<s>", "</s>"]

In [10]:
#设置训练器并加载特殊token
trainer = trainers.BpeTrainer(
    vocab_size=6400,
    special_tokens=special_tokens,
    show_progress=True,
    initial_alphabet=pre_tokenizers.ByteLevel.alphabet()
)

In [ ]:
#训练tokenizer
tokenizer.train_from_iterator(texts,trainer=trainer)